based on  
https://github.com/langchain-ai/langchain/blob/master/cookbook/self_query_hotel_search.ipynb

## 데이터전처리

In [2]:
import pandas as pd

In [ ]:
data = pd.read_csv('/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu.csv')

In [ ]:
# 결과 확인
data.head()

,메뉴 이름,카테고리,신메뉴,메뉴 설명,식재료,중량,열량,가격,세트메뉴 가능여부,세트가격
0,맥크리스피 스리라차 마요,버거,True,빠삭한 100% 통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛,"난류,우유,대두,밀,토마토,닭고기,쇠고기",289.0,663.0,8600,True,10500.0
1,베토디 스리라차 마요,버거,True,베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함!,"난류,우유,대두,밀,돼지고기,토마토,쇠고기",251.0,621.0,7600,True,9500.0
2,맥스파이시 상하이 버거,버거,False,쌀가루가 더해져 더 바삭해진 100% 닭가슴살 패티에 아삭아삭한 양상추와 신선한 토...,"난류,대두,밀,토마토,닭고기",246.0,501.0,6300,True,8400.0
3,토마토 치즈 비프 버거,버거,True,신선한 토마토와 고소한 치즈버거의 조화,"난류,우유,대두,밀,토마토,쇠고기",200.0,403.0,4600,True,6900.0
4,더블 쿼터파운더 치즈,버거,False,"좋아하는건 더 많이 즐기시라고, 두 배 이상 커진 1/4 파운드 비프 패티가 두 장...","우유,대두,밀,토마토,쇠고기",275.0,770.0,8200,True,10400.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   메뉴 이름      27 non-null     object 
 1   카테고리       27 non-null     object 
 2   신메뉴        27 non-null     bool   
 3   메뉴 설명      27 non-null     object 
 4   식재료        22 non-null     object 
 5   중량         22 non-null     float64
 6   열량         22 non-null     float64
 7   가격         27 non-null     int64  
 8   세트메뉴 가능여부  22 non-null     object 
 9   세트가격       21 non-null     float64
dtypes: bool(1), float64(3), int64(1), object(5)
memory usage: 2.0+ KB


In [ ]:
# 데이터프레임의 열 이름 확인
print("데이터프레임의 열 이름:")
print(data.columns)

# 각 열의 데이터 타입 확인
print("\n각 열의 데이터 타입:")
print(data.dtypes)


데이터프레임의 열 이름:
Index(['메뉴 이름 ', '카테고리', '신메뉴', '메뉴 설명 ', '식재료', '중량', '열량', '가격 ',
       '세트메뉴 가능여부', '세트가격 '],
      dtype='object')

각 열의 데이터 타입:
메뉴 이름         object
카테고리          object
신메뉴             bool
메뉴 설명         object
식재료           object
중량           float64
열량           float64
가격             int64
세트메뉴 가능여부     object
세트가격         float64
dtype: object


In [ ]:
data.columns = ['menuname', 'category', 'newmenu', 'description', 'ingredients', 'grams', 'kcals', 'burgerprice ','setmenu', 'setprice ']

In [ ]:
data.head()

,menuname,category,newmenu,description,ingredients,grams,kcals,burgerprice,setmenu,setprice
0,맥크리스피 스리라차 마요,버거,True,빠삭한 100% 통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛,"난류,우유,대두,밀,토마토,닭고기,쇠고기",289.0,663.0,8600,True,10500.0
1,베토디 스리라차 마요,버거,True,베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함!,"난류,우유,대두,밀,돼지고기,토마토,쇠고기",251.0,621.0,7600,True,9500.0
2,맥스파이시 상하이 버거,버거,False,쌀가루가 더해져 더 바삭해진 100% 닭가슴살 패티에 아삭아삭한 양상추와 신선한 토...,"난류,대두,밀,토마토,닭고기",246.0,501.0,6300,True,8400.0
3,토마토 치즈 비프 버거,버거,True,신선한 토마토와 고소한 치즈버거의 조화,"난류,우유,대두,밀,토마토,쇠고기",200.0,403.0,4600,True,6900.0
4,더블 쿼터파운더 치즈,버거,False,"좋아하는건 더 많이 즐기시라고, 두 배 이상 커진 1/4 파운드 비프 패티가 두 장...","우유,대두,밀,토마토,쇠고기",275.0,770.0,8200,True,10400.0


In [ ]:
# 한글과 영어만 남기고 나머지 문자 제거하는 함수 정의
def clean_text(text):
    import re
    return re.sub(r'[^가-힣a-zA-Z\s]', '', str(text))

def clean_text_num(text):
    import re
    return re.sub(r'[^가-힣a-zA-Z0-9\s]', '', str(text))

# 데이터프레임의 특정열에 clean_text 함수 적용
data[['category', 'newmenu', 'description']] = data[['category', 'newmenu', 'description']].map(clean_text)
data['menuname'] = data['menuname'].map(clean_text_num)


In [ ]:
# xao remove
def clean_xao(text):
    return text.replace(u'\xa0', u' ')

# 데이터프레임의 특정열에 clean_text 함수 적용
data[['menuname', 'category', 'newmenu', 'description']] = data[['menuname', 'category', 'newmenu', 'description']].map(clean_xao)

In [ ]:
data.head()

,menuname,category,newmenu,description,ingredients,grams,kcals,burgerprice,setmenu,setprice
0,맥크리스피 스리라차 마요,버거,True,빠삭한 통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛,"난류,우유,대두,밀,토마토,닭고기,쇠고기",289.0,663.0,8600,True,10500.0
1,베토디 스리라차 마요,버거,True,베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함,"난류,우유,대두,밀,돼지고기,토마토,쇠고기",251.0,621.0,7600,True,9500.0
2,맥스파이시 상하이 버거,버거,False,쌀가루가 더해져 더 바삭해진 닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지...,"난류,대두,밀,토마토,닭고기",246.0,501.0,6300,True,8400.0
3,토마토 치즈 비프 버거,버거,True,신선한 토마토와 고소한 치즈버거의 조화,"난류,우유,대두,밀,토마토,쇠고기",200.0,403.0,4600,True,6900.0
4,더블 쿼터파운더 치즈,버거,False,좋아하는건 더 많이 즐기시라고 두 배 이상 커진 파운드 비프 패티가 두 장 그램육...,"우유,대두,밀,토마토,쇠고기",275.0,770.0,8200,True,10400.0


In [ ]:
# '버거' 단어가 없는 메뉴 이름에 '버거' 추가 (category가 '버거'인 경우에만)
def add_burger(row):
    if '버거' in row['category'] and '버거' not in row['menuname']:
        return row['menuname'] + ' 버거'
    return row['menuname']

# 'map' 대신 'apply' 사용
data['menuname'] = data.apply(add_burger, axis=1)

In [ ]:
data['menuname'].value_counts().index.tolist()

['맥크리스피 스리라차 마요 버거',
 '불고기 버거',
 '후렌치후라이 미디움',
 '후렌치후라이 스몰',
 '탄산 라지',
 '탄산 미디움',
 '햄버거',
 '트리플 치즈버거',
 '더블 치즈버거',
 '치즈버거',
 '베이컨 토마토 디럭스 버거',
 '슈비 버거',
 '슈슈 버거',
 '에그 불고기 버거',
 '베토디 스리라차 마요 버거',
 '더블 불고기 버거',
 '맥치킨 버거',
 '맥치킨 모짜렐라 버거',
 '1955 버거',
 '맥크리스피 클래식 버거',
 '맥크리스피 디럭스 버거',
 '빅맥 버거',
 '쿼터파운더 치즈 버거',
 '더블 쿼터파운더 치즈 버거',
 '토마토 치즈 비프 버거',
 '맥스파이시 상하이 버거',
 '후렌치후라이 라지']

In [ ]:
# data.to_csv('MD_menu_cleaned.csv', index=False)

### 결측치 확인

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   menuname      27 non-null     object 
 1   category      27 non-null     object 
 2   newmenu       27 non-null     bool   
 3   description   27 non-null     object 
 4   ingredients   22 non-null     object 
 5   grams         22 non-null     float64
 6   kcals         22 non-null     float64
 7   burgerprice   27 non-null     int64  
 8   setmenu       22 non-null     object 
 9   setprice      21 non-null     float64
dtypes: bool(1), float64(3), int64(1), object(5)
memory usage: 2.0+ KB


In [ ]:
data.isna().sum()

menuname        0
category        0
newmenu         0
description     0
ingredients     5
grams           5
kcals           5
burgerprice     0
setmenu         5
setprice        6
dtype: int64

## LLM 연결

In [1]:
import pandas as pd
data = pd.read_csv('/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv')

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
res = model.predict(
    "Below is a table with information about McDonald's menu. "
    "Return a JSON list with an entry for each column. Each entry should have "
    '{"name": "column name", "description": "column description", "type": "column data type"}'
    f"\n\n{data.head()}\n\nJSON:\n"
)

/tmp/ipykernel_18002/4102784480.py:4: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  res = model.predict(


In [24]:
import json

# 문서에서 사용할 속성 정보
attribute_information = json.loads(res)


In [30]:
attribute_info = attribute_information.copy()

In [31]:
data.nunique()[data.nunique() < 40]

menuname        27
category         3
newmenu          2
description     24
ingredients     12
grams           21
kcals           21
burgerprice     22
setmenu          2
setprice        17
dtype: int64

In [32]:
attribute_info[0]["description"] += (
    f". Valid values are {sorted(data['menuname'].value_counts().index.tolist())}"
)

In [33]:
attribute_info[1]["description"] += (
    f". Valid values are {sorted(data['category'].value_counts().index.tolist())}"
)

In [34]:
# 'ingredients' 열의 모든 고유한 값을 추출
unique_ingredients = set()
data['ingredients'].dropna().apply(lambda x: unique_ingredients.update(x.split(',')))

# 'ingredients' 항목에 대한 설명 추가
attribute_info[4]["description"] += (
    f". Valid values are {sorted(unique_ingredients)}"
)

In [35]:
attribute_info[0]["description"] += (
    ". Note: Only use the 'eq' operator if a specific menu name is mentioned. If '햄버거' is mentioned, ask if it is a name of a burger or a category."
)

In [36]:
attribute_info[1]["description"] += (
    ". Note: Only use the 'eq' operator if a specific category is mentioned."
)

In [38]:
attribute_info[4]["description"] += (
    ". Note: Only use the 'in' or 'nin operator if a specific ingredient is mentioned. If a general ingredient is mentioned, include all relevant ingredients in filter."
)

In [39]:
attribute_info

[{'name': 'menuname',
  'description': "name of the menu item. Valid values are ['1955 버거', '더블 불고기 버거', '더블 치즈버거', '더블 쿼터파운더 치즈 버거', '맥스파이시 상하이 버거', '맥치킨 모짜렐라 버거', '맥치킨 버거', '맥크리스피 디럭스 버거', '맥크리스피 스리라차 마요 버거', '맥크리스피 클래식 버거', '베이컨 토마토 디럭스 버거', '베토디 스리라차 마요 버거', '불고기 버거', '빅맥 버거', '슈비 버거', '슈슈 버거', '에그 불고기 버거', '치즈버거', '쿼터파운더 치즈 버거', '탄산 라지', '탄산 미디움', '토마토 치즈 비프 버거', '트리플 치즈버거', '햄버거', '후렌치후라이 라지', '후렌치후라이 미디움', '후렌치후라이 스몰']. Note: Only use the 'eq' operater if a specific menu name is mentioned. If '햄버거' is mentioned, ask if it is a name of burger or category of burger.. Valid values are ['1955 버거', '더블 불고기 버거', '더블 치즈버거', '더블 쿼터파운더 치즈 버거', '맥스파이시 상하이 버거', '맥치킨 모짜렐라 버거', '맥치킨 버거', '맥크리스피 디럭스 버거', '맥크리스피 스리라차 마요 버거', '맥크리스피 클래식 버거', '베이컨 토마토 디럭스 버거', '베토디 스리라차 마요 버거', '불고기 버거', '빅맥 버거', '슈비 버거', '슈슈 버거', '에그 불고기 버거', '치즈버거', '쿼터파운더 치즈 버거', '탄산 라지', '탄산 미디움', '토마토 치즈 비프 버거', '트리플 치즈버거', '햄버거', '후렌치후라이 라지', '후렌치후라이 미디움', '후렌치후라이 스몰']. Note: Only use the 'eq' operator if a specific menu n

In [40]:
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt, # 자연어 쿼리의 구조를 정의하는 프롬프트를 생성하는 함수
    load_query_constructor_runnable, # 실제로 자연어 쿼리를 처리하고 실행할 수 있는 체인을 로드하는 함수
)

In [41]:
# 쿼리를 구성할 때 참조할 문서의 내용을 전달
doc_contents = "Detailed description of McDonald's menu"
prompt = get_query_constructor_prompt(doc_contents, attribute_info)
print(prompt.format(query="{query}"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

In [42]:
chain = load_query_constructor_runnable(
    ChatOpenAI(model="gpt-3.5-turbo", temperature=0), doc_contents, attribute_info
)

In [43]:
# 이 결과물이 filter, limit이 아무것도 뜨지 않는 이유:
# 1. 데이터셋에 굴이 들어간 버거가 없을 가능성이 높습니다.
# 2. 쿼리가 정확히 매칭되는 항목을 찾지 못했을 수 있습니다.
# 3. 모델이 쿼리를 정확히 해석하지 못했을 수 있습니다.

# 확인을 위해 다음과 같은 작업을 수행할 수 있습니다:
print("데이터셋 내 '굴' 포함 여부:")
print(data['ingredients'].str.contains('굴').any())

print("\n모든 고유 재료:")
print(unique_ingredients)

print("\n'굴'이 포함된 메뉴 (있다면):")
print(data[data['ingredients'].str.contains('굴', na=False)])

데이터셋 내 '굴' 포함 여부:
True

모든 고유 재료:
{'난류', '새우', '대두', '닭고기', '돼지고기', '조개', '토마토', '쇠고기', '밀', '우유', '굴'}

'굴'이 포함된 메뉴 (있다면):
     menuname category  newmenu  \
12  더블 불고기 버거      버거     False   
13  에그 불고기 버거      버거     False   
14     불고기 버거      버거     False   
15      슈슈 버거      버거     False   
16      슈비 버거      버거     False   

                                          description  \
12  진한 불고기 소스의 패티가 두 장 여기에 고소한 치즈와 마요네즈 신선한 양상추를 곁...   
13                   불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다    
14  한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상...   
15                                  탱글한 통새우살 가득 슈슈 버거   
16                           탱글한 통새우살에 비프 패티를 더해 푸짐하게   

                ingredients  grams  kcals  burgerprice  setmenu  setprice   
12     난류,우유,대두,밀,돼지고기,조개,굴  237.0  635.0          5300    True     7500.0  
13        난류,대두,밀,돼지고기,조개,굴  222.0  491.0          5100    True     7300.0  
14        난류,대두,밀,돼지고기,조개,굴  167.0  409.0          4200    True     6600.0  
15  난류,우유,대두,밀

In [44]:
chain.invoke({"query": "굴이 들어간 버거는 뭐가있어?"})

StructuredQuery(query='굴', filter=Comparison(comparator=<Comparator.IN: 'in'>, attribute='ingredients', value=['굴']), limit=None)

In [45]:
chain.invoke({"query": "닭고기가 들어간 버거는 뭐가있어?"})

StructuredQuery(query='닭고기', filter=Comparison(comparator=<Comparator.IN: 'in'>, attribute='ingredients', value=['굴']), limit=None)

In [79]:
chain.invoke({"query": "치킨버거 종류는 뭐가 있어?"})

StructuredQuery(query='치킨버거', filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='category', value='버거'), limit=None)

갑분 굴...

In [46]:
chain.invoke({"query": "버거 중 신메뉴는 뭐가있어?"})

StructuredQuery(query='신메뉴', filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='newmenu', value=True), limit=None)

In [47]:
chain.invoke({"query": "열량이 제일 낮은 버거는 뭐야?"})

StructuredQuery(query='lowest calorie burger', filter=None, limit=None)

In [48]:
chain.invoke({"query": "what is the lowest calorie burger?"})

StructuredQuery(query=' ', filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='category', value='버거'), limit=None)

In [49]:
chain.invoke({"query": "recommend me a burger with spicy taste"})

StructuredQuery(query='spicy taste', filter=None, limit=None)

## embedding 파일 연결하기

In [34]:
data.head()

,menuname,category,newmenu,description,ingredients,grams,kcals,burgerprice,setmenu,setprice
0,맥크리스피 스리라차 마요 버거,버거,True,빠삭한 통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛,"난류,우유,대두,밀,토마토,닭고기,쇠고기",289.0,663.0,8600,True,10500.0
1,베토디 스리라차 마요 버거,버거,True,베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함,"난류,우유,대두,밀,돼지고기,토마토,쇠고기",251.0,621.0,7600,True,9500.0
2,맥스파이시 상하이 버거,버거,False,쌀가루가 더해져 더 바삭해진 닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지...,"난류,대두,밀,토마토,닭고기",246.0,501.0,6300,True,8400.0
3,토마토 치즈 비프 버거,버거,True,신선한 토마토와 고소한 치즈버거의 조화,"난류,우유,대두,밀,토마토,쇠고기",200.0,403.0,4600,True,6900.0
4,더블 쿼터파운더 치즈 버거,버거,False,좋아하는건 더 많이 즐기시라고 두 배 이상 커진 파운드 비프 패티가 두 장 그램육...,"우유,대두,밀,토마토,쇠고기",275.0,770.0,8200,True,10400.0


In [72]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [73]:
docs = []
for _, menu in data.fillna("").iterrows():
    doc = Document(
        page_content=json.dumps(menu.to_dict(), indent=2),
        metadata=menu.to_dict()
    )
    docs.append(doc)

In [74]:
docs

[Document(metadata={'menuname': '맥크리스피 스리라차 마요 버거', 'category': '버거 ', 'newmenu': True, 'description': '빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛 ', 'ingredients': '난류,우유,대두,밀,토마토,닭고기,쇠고기', 'grams': 289.0, 'kcals': 663.0, 'burgerprice ': 8600, 'setmenu': True, 'setprice ': 10500.0}, page_content='{\n  "menuname": "\\ub9e5\\ud06c\\ub9ac\\uc2a4\\ud53c \\uc2a4\\ub9ac\\ub77c\\ucc28 \\ub9c8\\uc694 \\ubc84\\uac70",\n  "category": "\\ubc84\\uac70 ",\n  "newmenu": true,\n  "description": "\\ube60\\uc0ad\\ud55c  \\ud1b5\\ub2ed\\ub2e4\\ub9ac\\uc0b4 \\ucf00\\uc774\\uc900 \\ud328\\ud2f0\\uc5d0 \\uc2a4\\ub9ac\\ub77c\\ucc28 \\ub9c8\\uc694 \\uc18c\\uc2a4\\ub97c \\ub354\\ud574 \\ub9e4\\ucf64 \\uace0\\uc18c\\ud55c \\ub9db ",\n  "ingredients": "\\ub09c\\ub958,\\uc6b0\\uc720,\\ub300\\ub450,\\ubc00,\\ud1a0\\ub9c8\\ud1a0,\\ub2ed\\uace0\\uae30,\\uc1e0\\uace0\\uae30",\n  "grams": 289.0,\n  "kcals": 663.0,\n  "burgerprice ": 8600,\n  "setmenu": true,\n  "setprice ": 10500.0\n}'),
 Document(metadata={'menunam

In [75]:
# 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    docs, OpenAIEmbeddings(model="text-embedding-3-large")
)

In [76]:
from langchain.retrievers import SelfQueryRetriever

retriever = SelfQueryRetriever(
    query_constructor=chain, vectorstore=vectorstore, verbose=True
)


In [77]:
results = retriever.invoke(
    "굴이 들어가지 않는 버거는 뭐가 있어?"
)
for res in results:
    # ensure_ascii=False로 한글을 정상 출력
    print(json.dumps(json.loads(res.page_content), indent=2, ensure_ascii=False))
    print("\n" + "-" * 20 + "\n")


ValueError: Received disallowed comparator nin. Allowed comparators are [<Comparator.EQ: 'eq'>, <Comparator.NE: 'ne'>, <Comparator.GT: 'gt'>, <Comparator.GTE: 'gte'>, <Comparator.LT: 'lt'>, <Comparator.LTE: 'lte'>]

In [78]:
results = retriever.invoke(
    "열량이 제일 낮은 버거는 뭐야?"
)
for res in results:
    # ensure_ascii=False로 한글을 정상 출력
    print(json.dumps(json.loads(res.page_content), indent=2, ensure_ascii=False))
    print("\n" + "-" * 20 + "\n")


{
  "menuname": "슈슈 버거",
  "category": "버거 ",
  "newmenu": false,
  "description": "탱글한 통새우살 가득 슈슈 버거",
  "ingredients": "난류,우유,대두,밀,토마토,새우,쇠고기,굴",
  "grams": 224.0,
  "kcals": 409.0,
  "burgerprice ": 5500,
  "setmenu": true,
  "setprice ": 7300.0
}

--------------------

{
  "menuname": "에그 불고기 버거",
  "category": "버거 ",
  "newmenu": false,
  "description": "불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다 ",
  "ingredients": "난류,대두,밀,돼지고기,조개,굴",
  "grams": 222.0,
  "kcals": 491.0,
  "burgerprice ": 5100,
  "setmenu": true,
  "setprice ": 7300.0
}

--------------------

{
  "menuname": "치즈버거",
  "category": "버거 ",
  "newmenu": false,
  "description": "고소하고 부드러운 치즈와  순 쇠고기 패티 맥도날드만의 심플한 클래식 치즈버거 ",
  "ingredients": "우유,대두,밀,토마토,쇠고기",
  "grams": 115.0,
  "kcals": 318.0,
  "burgerprice ": 3600,
  "setmenu": true,
  "setprice ": 6300.0
}

--------------------

{
  "menuname": "슈비 버거",
  "category": "버거 ",
  "newmenu": false,
  "description": "탱글한 통새우살에 비프 패티를 더해 푸짐하게",
  "ingredients": "난류,우유,대두,밀,토마토,새우,